<a href="https://colab.research.google.com/github/RichardHarwood/3D_Unet_Plant_Roots/blob/main/Analysis_Examples_Incorparating_3D_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wget
!pip install open3d
!pip install vedo
#!pip install stackview ipycanvas==0.11
import os
!git clone https://github.com/wolny/pytorch-3dunet.git
%cd pytorch-3dunet
!python setup.py install
!pip install imagecodecs
!pip install skan
!pip install porespy
!pip install pypardiso
!pip install plotly

In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#need to set appropriate  home folder
import os.path
from os import path
if path.exists('/content/drive/MyDrive/3D_UNET_analysis_eg/') == False:
  os.mkdir('/content/drive/MyDrive/3D_UNET_analysis_eg/')
home_folder = "/content/drive/MyDrive/3D_UNET_analysis_eg/"

In [ ]:
#Create Config Folder
if path.exists(home_folder+'config_files') == False:
  os.mkdir(home_folder+'config_files')
config_folder = home_folder+'config_files/'

In [ ]:
#Create Checkpoint Folder
if path.exists(home_folder+'checkpoint_dir') == False:
  os.mkdir(home_folder+'checkpoint_dir')
checkpoint_dir = home_folder+'checkpoint_dir/'

In [ ]:
#Create Folder to store images we want to segment
if path.exists(home_folder+'segment_this_folder') == False:
  os.mkdir(home_folder+'segment_this_folder')
imgs_to_seg_dir = home_folder+'segment_this_folder/'

## Download image to test workflow on
can be slow depending on image, using a very small one here to illustrate the workflow

In [ ]:
import wget
import wget
if os.path.isfile(imgs_to_seg_dir+'root_unet_eg.tif'):
  print("image already download")
else:
  wget.download("https://zenodo.org/records/14189395/files/"+"recon_W1_WW_S1_R3.tif"+"?download=1", out=imgs_to_seg_dir+'root_unet_eg.tif')



In [ ]:
#Convert to H5
import skimage
import imagecodecs
import h5py
from scipy.ndimage import zoom
#need to reduce image due to RAM depending on what resources you have
scale_factor=0.2
img=skimage.io.imread(imgs_to_seg_dir+'root_unet_eg.tif')
img=zoom(img, (scale_factor,scale_factor, scale_factor), order=1)
hf = h5py.File(imgs_to_seg_dir+"root_unet_eg.h5", 'w')
hf.create_dataset('raw', data=img)
hf.close()



#Downlaod model to segment image


In [ ]:
if os.path.isfile(checkpoint_dir+'3D_Unet_Root_Model.pytorch'):
  print("model already download")
else:
  wget.download("https://zenodo.org/records/13958667/files/"+"3D_Unet_Root_Model.pytorch"+"?download=1", out=checkpoint_dir+'3D_Unet_Root_Model.pytorch')


In [ ]:
model_name = "UNet3D"
in_channels = 1
out_channels= 1
layer_order = "gcr"
f_maps = [32,
          64,
          128,
          256]
num_groups = 8
final_sigmoid = True
is_segmentation = True
batch_size= 2  #important for gpu memory
num_workers= 2
raw_internal_path= "raw"
label_internal_path= "label"
weight_internal_path= None
######################
slice_builder_name= "SliceBuilder"
slice_builder_name_predict = "SliceBuilder"
patch_shape= [40, 170, 170]  #Change depending on GPU memory
stride_shape= [20, 40, 40]   #Change depending on GPU memory
halo_shape= [16, 32, 32]
threshold = 0.01
slack_acceptance = 0.01
name_transformer= "Standardize"
name_transformer_label= "BlobsToMask"
append_label= False
boundary= False
ToTensor_name = "ToTensor"
expand_dims_false= False
expand_dims_true= True
#####################
predictor_name='StandardPredictor'
#################################
file_paths_test= imgs_to_seg_dir
#############################
model_path=checkpoint_dir+'3D_Unet_Root_Model.pytorch'

In [ ]:
import yaml
test_config_yaml ={
    "model_path": model_path,
    "model": {
        "name": model_name,
        "in_channels": in_channels,
        "out_channels": out_channels,
        "layer_order": layer_order,
        "f_maps": f_maps,
        "num_groups": num_groups,
        "final_sigmoid": final_sigmoid,
        "is_segmentation": is_segmentation
    },
    "predictor": {
        "name": predictor_name
    },
    "loaders": {
        "batch_size": batch_size,
        "raw_internal_path": raw_internal_path,
        "num_workers": num_workers,
        "test": {
            "file_paths": [
                file_paths_test
            ],
            "slice_builder": {
                "name": slice_builder_name_predict,
                "patch_shape": patch_shape,
                "stride_shape": patch_shape,
                "halo_shape": halo_shape
            },
            "transformer": {
                "raw": [
                    {
                        "name": name_transformer
                    },
                    {
                        "name": ToTensor_name,
                        "expand_dims": expand_dims_true
                    }
                ]
            }
        }
    }
}

with open(config_folder+'predict.yaml', 'w') as yaml_file:
    yaml.dump(test_config_yaml, yaml_file, default_flow_style=False, sort_keys=False)


In [ ]:
!predict3dunet --config /content/drive/MyDrive/3D_UNET_analysis_eg/config_files/predict.yaml

In [ ]:
import glob
import numpy as np
raw_data_path= file_paths_test
raw_file_names = [os.path.basename(x) for x in glob.glob(imgs_to_seg_dir+"*.h5")]
raw_file_names = [e for e in raw_file_names if "predictions" in e ]
#################################
if path.exists(home_folder+'processed/') == False:
  os.mkdir(home_folder+'processed/')
file_paths_processed= home_folder+'processed/'
#########################
#raw_file_names

UNIQUE_ID =np.unique(raw_file_names)
len(UNIQUE_ID)

In [ ]:
UNIQUE_ID

In [ ]:
#scale_factor=0.5
Z=6.5*2
Y=6.5*2
X=6.5*2

In [ ]:
import h5py
import cv2
import skimage
from skimage.filters import threshold_otsu, threshold_yen,threshold_li
from skimage.measure import label
from skimage.morphology import remove_small_objects
from vedo import *
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

for i in UNIQUE_ID:
    print(i)
    hf=h5py.File(raw_data_path+i)
    hf.keys()
    dataset_IMGs= hf['predictions']
    unet_prediction=dataset_IMGs[:]
    unet_prediction=unet_prediction[0,:,:,:]
    #unet_prediction=zoom(unet_prediction, (scale_factor,scale_factor, scale_factor), order=1)
    unet_prediction = cv2.normalize(unet_prediction, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    thresh = threshold_li(unet_prediction)
    root_binary = unet_prediction > thresh

    root_binary_clean=label(root_binary)
    min_size_coef=150000/4
    root_binary_clean = remove_small_objects(root_binary_clean, min_size=min_size_coef, connectivity=2)
    root_binary_clean = (root_binary_clean != 0).astype(bool)


    root_VOL = Volume(root_binary_clean, spacing=((1),(1),(1)))
    root_ISO = root_VOL.isosurface(1)
    root_ISO.write(file_paths_processed+i+"__root.stl")
    skimage.io.imsave(file_paths_processed+i+"_processed.tif", root_binary_clean, check_contrast=False)
    plt.imshow(root_binary_clean[30])

In [ ]:
#Stls love crashing colab for some reason (PyVitsa and Vedo) so just surface plot the root to inspect
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
binary_image=root_binary_clean.astype(int)
x, y, z = np.indices(binary_image.shape)
x_vals, y_vals, z_vals = x[binary_image == 1], y[binary_image == 1], z[binary_image == 1]
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_vals, y_vals, z_vals, c='r', marker='o')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Root Image Surface Plot')
plt.show()

#Example one: Extract the skeleton
https://skeleton-analysis.org/stable/index.html

In [ ]:
#package import not sorted correctly, kind of done for each example
import numpy as np
import skan
from skimage.data import binary_blobs
from skimage.morphology import skeletonize
import scipy.ndimage as ndi

#root_binary_clean_sml=zoom(root_binary_clean, (0.5,0.5, 0.5), order=0)
binary_skeleton = skeletonize(root_binary_clean)
skeleton = skan.Skeleton(binary_skeleton)

all_paths = [
        skeleton.path_coordinates(i)
        for i in range(skeleton.n_paths)
        ]

paths_table = skan.summarize(skeleton, separator='_')
paths_table['path_id'] = np.arange(skeleton.n_paths)
paths_table['random_path_id'] = np.random.default_rng().permutation(skeleton.n_paths)*6.6 #need to tweak this for vis to work
paths_table.head()


#Example two: Extract the pore network

In [ ]:
import porespy as ps
import openpnm as op
import scipy.ndimage as spim
from porespy.filters import find_peaks, trim_saddle_points, trim_nearby_peaks
from porespy.tools import randomize_colors
from scipy import ndimage as ndi
from skimage.segmentation import watershed
import seaborn as sns
#Dilate around the root to make a rhizosheath
diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
iternation_n=20
dilated = ndi.binary_dilation(root_binary_clean, diamond, iterations=iternation_n)
rhizosheath= root_binary_clean ^ dilated
#Get Pore Network
ct_data=zoom(img, (scale_factor,scale_factor, scale_factor), order=1)
pore_thresh = threshold_otsu(ct_data)
pore= ct_data < pore_thresh
#Get Pore Network in Rhizosheath
pore_rhizo=pore * rhizosheath

#######################################
sigma = 0.4
dt = spim.distance_transform_edt(input=pore_rhizo)
dt1 = spim.gaussian_filter(input=dt, sigma=sigma)
peaks = find_peaks(dt=dt)
print('Initial number of peaks: ', spim.label(peaks)[1])
peaks = trim_saddle_points(peaks=peaks, dt=dt1)
print('Peaks after trimming saddle points: ', spim.label(peaks)[1])
peaks = trim_nearby_peaks(peaks=peaks, dt=dt)
peaks, N = spim.label(peaks)
print('Peaks after trimming nearby peaks: ', N)
regions = watershed(image=-dt, markers=peaks, mask=dt > 0)
regions = randomize_colors(regions)
############################
mat_slice=int(len(rhizosheath)/2)
plt.figure(figsize= (5,5));
plt.imshow(rhizosheath[mat_slice]);
plt.title("rhizosheath");
plt.show();

import pandas as pd
import imageio
net = ps.networks.regions_to_network(regions*pore_rhizo, voxel_size=X)
pn = op.io.network_from_porespy(net)

pore_region_volume_df=pd.DataFrame(pn['pore.region_volume'])
throat_cons_df=pd.DataFrame(pn['throat.conns'])

#saves data for visualising, cant be donw in colab, check our parraview
#pore_rhizo = (np.swapaxes(pore_rhizo, 2, 0)) # This isjust some of ps.tools.align_image_with_openpnm(im), it works to align image with
imageio.volsave(file_paths_processed+'pore_networkfor_rhizosheath.tif', np.array(pore_rhizo, dtype=np.int8))
op.io.project_to_vtk(project=pn.project,filename=(file_paths_processed+"pore_network"))

#sns.kdeplot(data=pore_region_volume_df, x=0).set_title("Spead of region pore size");

fig, ax = plt.subplots(figsize=[5, 5])
#ax.imshow(slice_2d.T, cmap=plt.cm.bone);
op.visualization.plot_coordinates(ax=fig,
                                  network=pn,
                                  size_by=pn["pore.region_volume"],
                                  color_by=pn["throat.total_length"],
                                  markersize=50)
op.visualization.plot_connections(network=pn, ax=fig)
ax.set_title('Pore Network in the rhizosheath')
plt.show()



In [ ]:
#interact with the pore network
op.visualization.plot_notebook(network=pn, node_color=0, edge_color=0, node_size=1, node_scale=20, edge_scale=5, colormap='viridis')

#Example three: Visualize  root density
Pretty underwhelming with this example